In [ ]:
import os
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from scipy.cluster.hierarchy import dendrogram, linkage
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler, Normalizer
import numpy as np
import pandas as pd
import seaborn as sns
from scipy.cluster.hierarchy import fcluster, set_link_color_palette
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.metrics import silhouette_samples, silhouette_score
from sklearn.mixture import GaussianMixture as GM
import dc_stat_think as dcst
from scipy.stats import chi2_contingency
import dabest
sep = '/'
plt.style.use('ggplot')
plt.rcParams['axes.facecolor']='#f2efef'

In [ ]:
font = {'fontname':'Calibri'}

In [ ]:
recording = r'E:\CIT_WAY\dat_files\cat\csvs'
filename ='neuron_stats.csv'
figpath = r'C:\Users\ASUS\Desktop\Images\Pearson'
figname = 'Pearson with labels.png'

### Load data, separate into 2 dataframes for CIT and Saline groups

In [ ]:
df = pd.read_csv(os.sep.join([recording, filename]))
df.index.name = 'Cluster_ID'
df = df[pd.notnull(df['SW_troff'])]

In [ ]:
df_cit = df[(df['recording']=='Chronic_04_2018-08-05')|
            (df['recording']=='Chronic_30_2018-07-28')|
            (df['recording']=='Chronic_40_2018-08-13')|
            (df['recording']=='Chronic_41_2018-08-09')]
df_cit = df_cit[(df_cit['CV ISI'] != 0)&(df_cit['Firing Rate'] != 0)]

In [ ]:
df_saline = df[(df['recording']=='Chronic_03_2018-07-29')|
               (df['recording']=='Chronic_11_2018-08-04')|
               (df['recording']=='Chronic_13_2018_08_08')|
               (df['recording']=='Chronic_14_2018-08-14')|
               (df['recording']=='Chronic_31_2018-08-10')]
df_saline = df_saline[(df_saline['CV ISI'] != 0)&(df_saline['Firing Rate'] != 0)]

### Make a table with the counts of each neuron type in the two groups

In [ ]:
df_pearson=pd.concat([df_cit.groupby('category').count()['spike_cluster'],
                      df_saline.groupby('category').count()['spike_cluster']], axis=1)
df_pearson.columns = ['Chronic Citalopram', 'Control']
df_pearson.fillna(0, inplace=True)

In [ ]:
df_pearson

### Chi square test using scipy stats library

In [ ]:
chi2, p, dof, expected = chi2_contingency(df_pearson,correction=False)

# Bar charts comparing distribution of neuron types in 2 mice groups

### Conver the counts into percentage 

In [ ]:
cols = ['Chronic Citalopram','Control']
df_pearson[cols] = df_pearson[cols].div(df_pearson[cols].sum(axis=0), axis=1).multiply(100)

### Plot 

In [ ]:
VFast = df_pearson.loc['V.fast firing',:]
Fast = df_pearson.loc['fast firing',:]
Irreg = df_pearson.loc['slow irregular',:]
Reg = df_pearson.loc['slow regular',:]


barWidth = 0.35
r = [0,1]
names = ('Chronic Citalopram','Control')

plt.figure(figsize = (10,8))
         
plt.bar(r, Reg, color= '#ee4035', edgecolor='white', width=barWidth)
         
plt.bar(r, Irreg, bottom = Reg, color='#7bc043', edgecolor='white', width=barWidth)
         
plt.bar(r, Fast, bottom = [i+j for i,j in zip(Reg, Irreg)], color='#0392cf', edgecolor='white', width=barWidth)
         
plt.bar(r, VFast, bottom = [a+b+c for a,b,c in zip(Reg, Irreg, Fast)], color='#CCCC00', edgecolor='white', width=barWidth)
 
# Custom x axis
plt.xticks(r, names, **font, fontsize=25)
plt.yticks( **font, fontsize=20)
plt.ylabel('No. of neurons [%]', **font, fontsize=30)
plt.legend(['Slow Regular','Slow Irregular', 'Fast Firing', 'V.Fast Firing'], prop={'size':20}, )
#plt.savefig(figpath, dpi=600)
plt.show()
